# Cours TAL – Labo 7 : Classification de dépêches d’agence avec NLTK

**Objectifs**
L’objectif de ce labo est de réaliser des expériences de classification de documents avec la boîte à
outils NLTK sur le corpus de dépêches Reuters. Le labo est à effectuer en binôme. Le rendu sera un
notebook Jupyter présentant vos choix, votre code, vos résultats et les discussions. Le labo sera jugé
sur la qualité des expériences et sur la discussion des différentes options explorées.


# 1. Récupération des données

**Données :** les dépêches du corpus Reuters, tel qu’il est fourni par NLTK. Veuillez respecter la
division en données d’entraînement et données de test.

In [1]:
import nltk
from nltk.corpus import reuters
from nltk.stem import WordNetLemmatizer

# nltk.download('reuters')
# nltk.download('stopwords')
# nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

stop_words = set(nltk.corpus.stopwords.words('english'))

# Nous faisons le choix de traiter tous les mots en minuscule
documents = [(list(w.lower() for w in reuters.words(fileid)), category)
             for category in reuters.categories()
             for fileid in reuters.fileids(category)]

# documents une les stops words retirés
documents_no_stop_words = [(list(w.lower() for w in filter(lambda w: w.lower() not in stop_words, reuters.words(fileid))), category)
             for category in reuters.categories()
             for fileid in reuters.fileids(category)]

# documents après lemmatisation des mots
documents_lemmatized = [(list(w.lower() for w in map(lemmatizer.lemmatize, reuters.words(fileid))), category)
             for category in reuters.categories()
             for fileid in reuters.fileids(category)]

In [2]:
# Récupère la fréquence des mots en filtrant les mots de moins de 4 caractères (évite l'apprentissage sur des symboles ou abréviations)
import re
all_words = nltk.FreqDist(w.lower() for w in reuters.words() if re.match(r'^[a-z]{3,}$', w))

# On sélectionne les 2000 mots les plus fréquents comme features pour les classifieurs
word_features = list(all_words)[:2000]

In [11]:
import random

# Modifie les catégories des documents pour les rendre binaires
def documents_with_binary_category(documents, category):
    return [(d, category if c == category else 'other') for (d, c) in documents]

# Modifie les catégories des documents pour limiter les catégories à celles passées en paramètre
def documents_with_retrcted_categories(documents, categories):
    return [(d, category if category in categories else 'other') for (d, category) in documents]

# split un dataset en 80% train et 20% test pour chaque category
def split_dataset(documents):
    dataset = {}
    for (d, c) in documents:
        if c not in dataset:
            dataset[c] = []
        dataset[c].append(d)
    train_set = []
    test_set = []
    for c in dataset:
        random.shuffle(dataset[c])
        train_set += [(d, c) for d in dataset[c][int(len(dataset[c]) * 0.2):]]
        test_set += [(d, c) for d in dataset[c][:int(len(dataset[c]) * 0.2)]]
    return train_set, test_set

# Retourne un dictionnaire indiquant si une feature est présent dans le document
def document_contains_features(document):
    doc_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in doc_words)
    return features

def get_featuresets(documents):
    return [(document_contains_features(d), c) for (d,c) in documents]

# Retourne la précision, le rappel et le F-score pour un classifieur sur un test set donné
def print_scores_for_category(classifier, test_set, category):
    classifier_labels = classifier.classify_many(tup[0] for tup in test_set)
    reference_labels = [tup[1] for tup in test_set]
    
    docs_in_cat = len([l for l in reference_labels if l == category])
    docs_classified_in_cat = len([l for l in classifier_labels if l == category])
    docs_correctly_classified_in_cat = len([i for i in range(len(reference_labels)) if reference_labels[i] == category and classifier_labels[i] == category])
    
    recall = docs_correctly_classified_in_cat / docs_in_cat
    precision = docs_correctly_classified_in_cat / docs_classified_in_cat
    f_score = 2 * recall * precision / (recall + precision)
    
    print('Rappel:', recall)
    print('Précision:', precision)
    print('F-mesure:', f_score)

# 2. Classifieurs binaires

## 2.1 Classifieur binaire pour la catégorie 'money-fx'

### 2.1.1 Classifieur Bayésien naïf + lemmatisation

In [10]:
dataset_moneyfx_lemmatized = documents_with_binary_category(documents_lemmatized, 'money-fx')

train_set_moneyfx_lemmatized, test_set_moneyfx_lemmatized = split_dataset(get_featuresets(dataset_moneyfx_lemmatized))

classifier_moneyfx_lemmatized = nltk.NaiveBayesClassifier.train(train_set_moneyfx_lemmatized)

In [13]:
print_scores_for_category(classifier_moneyfx_lemmatized, test_set_moneyfx_lemmatized, 'money-fx')

Rappel: 0.7832167832167832
Précision: 0.3163841807909605
F-mesure: 0.4507042253521127


### 2.1.2 Classifieur Bayésien naïf + stops words retirés

In [14]:
dataset_moneyfx_no_stop_words = documents_with_binary_category(documents_no_stop_words, 'money-fx')

train_set_moneyfx_no_stop_words, test_set_moneyfx_no_stop_words = split_dataset(get_featuresets(dataset_moneyfx_no_stop_words))

classifier_moneyfx_no_stop_words = nltk.NaiveBayesClassifier.train(train_set_moneyfx_no_stop_words)

In [15]:
print_scores_for_category(classifier_moneyfx_no_stop_words, test_set_moneyfx_no_stop_words, 'money-fx')

Rappel: 0.6993006993006993
Précision: 0.32786885245901637
F-mesure: 0.4464285714285714


## 2.2 Classifieur binaire pour la catégorie 'grain'
### 2.2.1 Classifieur Bayésien naïf + lemmatisation

In [16]:
dataset_grain_lemmatized = documents_with_binary_category(documents_lemmatized, 'grain')

train_set_grain_lemmatized, test_set_grain_lemmatized = split_dataset(get_featuresets(dataset_grain_lemmatized))

classifier_grain_lemmatized = nltk.NaiveBayesClassifier.train(train_set_grain_lemmatized)

In [17]:
print_scores_for_category(classifier_grain_lemmatized, test_set_grain_lemmatized, 'grain')

Rappel: 0.75
Précision: 0.22250639386189258
F-mesure: 0.3431952662721894


### 2.2.2 Classifieur Bayésien naïf + stops words retirés

In [18]:
dataset_grain_no_stop_words = documents_with_binary_category(documents_no_stop_words, 'grain')

train_set_grain_no_stop_words, test_set_grain_no_stop_words = split_dataset(get_featuresets(dataset_grain_no_stop_words))

classifier_grain_no_stop_words = nltk.NaiveBayesClassifier.train(train_set_grain_no_stop_words)

In [19]:
print_scores_for_category(classifier_grain_no_stop_words, test_set_grain_no_stop_words, 'grain')

Rappel: 0.853448275862069
Précision: 0.24812030075187969
F-mesure: 0.3844660194174757


## 2.3 Classifieur binaire pour la catégorie 'nat-gas'
### 2.3.1 Classifieur Bayésien naïf + lemmatisation 

In [20]:
dataset_natgas_lemmatized = documents_with_binary_category(documents_lemmatized, 'nat-gas')

train_set_natgas_lemmatized, test_set_natgas_lemmatized = split_dataset(get_featuresets(dataset_natgas_lemmatized))

classifier_natgas_lemmatized = nltk.NaiveBayesClassifier.train(train_set_natgas_lemmatized)

In [21]:
print_scores_for_category(classifier_natgas_lemmatized, test_set_natgas_lemmatized, 'nat-gas')

Rappel: 0.5238095238095238
Précision: 0.05188679245283019
F-mesure: 0.09442060085836909


### 2.3.2 Classifieur Bayésien naïf + stops words retirés

In [23]:
dataset_natgas_no_stop_words = documents_with_binary_category(documents_no_stop_words, 'nat-gas')

train_set_natgas_no_stop_words, test_set_natgas_no_stop_words = split_dataset(get_featuresets(dataset_natgas_no_stop_words))

classifier_natgas_no_stop_words = nltk.NaiveBayesClassifier.train(train_set_natgas_no_stop_words)

In [24]:
print_scores_for_category(classifier_natgas_no_stop_words, test_set_natgas_no_stop_words, 'nat-gas')

Rappel: 0.6190476190476191
Précision: 0.1
F-mesure: 0.17218543046357618


### 2.4 Observations

Le dataset pré-traité en retirant les stops words semble donner de meilleurs résultats que le dataset pré-traité en lemmatisant les mots. Cela peut s'expliquer par le fait que les stops words sont des mots très fréquents qui n'apportent pas d'information utile pour la classification. En les retirant, on peut donc s'attendre à ce que le classifieur se concentre sur des mots plus significatifs pour la classification, car ils ne seront plus présent dans le featureset.

# 3. Classifieur multi-classes
## 3.1 Classifier Bayésien naïf + stop words retirés

In [13]:
documents_multi = documents_with_retrcted_categories(documents_no_stop_words, ['money-fx', 'grain', 'nat-gas'])

train_set_multi, test_set_multi = split_dataset(get_featuresets(documents_multi))

classifier_multi = nltk.NaiveBayesClassifier.train(train_set_multi)

In [14]:
print('Résultats pour la catégorie money-fx:')
print_scores_for_category(classifier_multi, test_set_multi, 'money-fx')

Résultats pour la catégorie money-fx:
Rappel: 0.7412587412587412
Précision: 0.3569023569023569
F-mesure: 0.4818181818181818


In [15]:
print('Résultats pour la catégorie grain:')
print_scores_for_category(classifier_multi, test_set_multi, 'grain')

Résultats pour la catégorie grain:
Rappel: 0.7844827586206896
Précision: 0.23697916666666666
F-mesure: 0.364


In [16]:
print('Résultats pour la catégorie nat-gas:')
print_scores_for_category(classifier_multi, test_set_multi, 'nat-gas')

Résultats pour la catégorie nat-gas:
Rappel: 0.6190476190476191
Précision: 0.1092436974789916
F-mesure: 0.1857142857142857


# 4. Conclusion

Comment ces scores du classifieur multi-classe se comparent-ils à ceux des trois classifieurs binaires plus haut ? Quelle est donc la meilleure stratégie de classification ?

Les scores du classifieur multi-classe sont à peine plus élevé que ceux des classifieurs binaires. Cela peut s'expliquer par le fait que lors de la classification binaires, certains mots ont une probabilité similaire de se retrouver dans la classe A ou B. En rajoutant une troisième classe C, celle-ci peut définir la classe parfaite pour le mot qui tiraillait la classe A et B.

Dans notre situation, la meilleure stratégie de classification réside dans une classifieur multi-classe pour un ensemble de documents pré-traités en retirant les stop-words.